# Data Science Curriculum

In this curriculum, we'll be exploring how to build a data science project. Our end goal is a simple interactive graphic that is able to predict how much precipitation we will receive on a certain day given some information about the rest of the weather (temperature, humidity, etc.) on that day. Along the way, we'll see how to scrape, model, and visualize data; we'll be using Python for this project. Let's get started!

# Level 1: Scraping Data

In this level, we'll be obtaining the data that we hope to analyze later; this is an often overlooked part of data science, but it is crucial because errors in obtaining the data can cause problems for further analysis.

Data can come in many different forms and originate from many sources; for example, you could imagine collecting the 5 most recent posts all of your friends on Facebook have made. Each of these posts would be considered as one data point; we can observe different things about these data (continuing the Facebook example, the number of words in the post, whether the post contains a photo, what time it was posted, etc.), and these are called *variables*. Often times, we want to observe something about a data point given certain information about the data point; in the Facebook example, we could try to predict whether there is a photo in the post based on other information about the post.

For this project, we'll be analyzing data from [Wunderground](http://www.wunderground.com/); this is a website that contains information about the weather. What we'll want to do is assemble a dataset of weather over the past couple of years to see if we can predict the amount of precipitation on a certain day based on other information about that day. 

This brings us to our first important lesson: *obtaining the data must be scalable*. Let's think about how we can assemble such a dataset; one way is to navigate the site ourselves for each day we want to analyze while recording the variables we wish to analyze. Unfortunately, this won't work if we want to analyze a lot of data because it'll take a long time to collect all of the data needed. We're lucky because we can in fact automate the collection of this data, meaning it'll take much less time.

The way we'll collect the data is by a process called **web scraping**. It turns out that a lot of important and valuable information is just located on webpages. We can programatically find the data we're looking for and then extract exactly the information we want.

To start collecting our data, let's look at a sample webpage that we may extract information from:

![Wunderground Screenshot](wunderground.png)

This webpage come from the following link: "http://www.wunderground.com/history/airport/KNYC/2016/1/1/DailyHistory.html". It looks like this page organizes the information well, which is good for us! It clearly denotes where the temperature for the day is located. Let's dig into this a little more by exploring how the webpage is presenting this information.

![Inspect Screenshot](inspect.png)

Let's right click on the 38$^\circ$, and then press "Inspect". This will allow us to look at the actual HTML code that generated the webpage we're looking at. We can then right click where it says tbody and press "Edit as HTML".

![Edit as HTML Screenshot](edit-as-html.png)

This should show you the code of this table, and the first lines should look like this:
```
<tbody>
		<tr>
		<td class="history-table-grey-header">Temperature</td>
		<td colspan="3" class="history-table-grey-header">&nbsp;</td>
		</tr>
		<tr>
		<td class="indent"><span>Mean Temperature</span></td>
		<td>
  <span class="wx-data"><span class="wx-value">38</span><span class="wx-unit">&nbsp;°F</span></span>
</td>
		<td>
  <span class="wx-data"><span class="wx-value">33</span><span class="wx-unit">&nbsp;°F</span></span>
</td>
		<td>&nbsp;</td>
		</tr>
		<tr>
		<td class="indent"><span>Max Temperature</span></td>
		<td>
  <span class="wx-data"><span class="wx-value">42</span><span class="wx-unit">&nbsp;°F</span></span>
</td>
		<td>
  <span class="wx-data"><span class="wx-value">39</span><span class="wx-unit">&nbsp;°F</span></span>
</td>
		<td>
  <span class="wx-data"><span class="wx-value">62</span><span class="wx-unit">&nbsp;°F</span></span>
(1966)</td>
		</tr>
```

Here, we're seeing exactly how the information is being presented to the browser. At this level, it may be a little tough to see, but essentially, each row of the table (starts with `<tr>` and ends with `</tr>` has more information about the day that we may be interested in.) This organization means that we can write a program that will extract the data we want. Awesome!

Now, let's think more about what scope of data we want; we probably want at least a couple of years' worth of data. Let's say that we're interested in data between January 1, 2013 and December 31, 2015. Now, we just need to write code that will allow us to get all of the links to the data we want. If you remember from above, the link was "http://www.wunderground.com/history/airport/KNYC/2016/1/1/DailyHistory.html" for January 1<sup>st</sup>, 2016. Thus, it seems likely that we only need to substitute the year, month, and day that we want data for, which is convenient. Let's get into some Python code.

In [ ]:
start_year = 2013
end_year = 2015
days_per_month = {1: 31, 2: 28, 3: 31, 4: 30,
                     5: 31, 6: 30, 7: 31, 8: 31,
                     9: 30, 10: 31, 11: 30, 12: 31}

link_format = "http://www.wunderground.com/history/airport/KNYC/{}/{}/{}/DailyHistory.html"
links = [link_format.format(year, month, day)
         for year in range(start_year, end_year + 1)
         for month in range(1, 13)     # 1 - 12 inclusive
         for day in range(1, days_per_month[month] + 1)]

print(len(links))
print("\n".join(links[:5]))

It looks like our code is working! As a sanity check, 365 * 3 = 1095, which means we have the expected number of links. Let's start by downloading all of these webpages to our computer to make the process of extracting the information later on. We'll be using the `requests` package to download the webpage.

(Note: this will take a while, and that's okay. Shouldn't be more than 10 or so minutes!)

In [ ]:
import requests
import os.path

for i, link in enumerate(links):
    if i % 50 == 0:
        print("Done with %d.." % i)
        
    fname = "{}.html".format(i)
    if os.path.isfile(fname):
        continue
    with open(fname, "w") as fout:
        r = requests.get(link)
        fout.write(r.text)

Great, now we have all the data stored locally on our computer! This was done so that analyzing it won't take as much time since all the data is local rather than on the webpage.

Let's try exploring one of the HTML pages using Beautiful Soup.

In [ ]:
from bs4 import BeautifulSoup

with open("0.html") as fin:
    soup = BeautifulSoup(fin.read(), "html.parser")

Using Beautiful Soup, we can look for particular things on different pages. For example, we can look for the links (`a` tags) on the page.

In [ ]:
all_as = soup.find_all('a')
for i in range(5):
    print(all_as[-i])
    print()

For the specific data we're looking for, it's all inside of a `table` with `historyTable` as the id. We can also search by id with BeautifulSoup.

In [ ]:
main_table = soup.find(id='historyTable')

Now we can look for `tr`'s within this table specifically.

In [ ]:
rows = main_table.find_all('tr')
print(len(rows))
for i in range(3):
    print(rows[i])
    print()

Once we have an actual row, there's lots of information we can extract. Let's try it with one of the rows.

(We're only interested in the first two cells because those are the row name and value on that day.)

In [ ]:
row = rows[2]
for cell in row.find_all('td'):
    print(cell)
    print()
row_name = row.find_all('td')[0].text.strip()  # Get rid of extra whitespace
row_value = row.find_all('td')[1].text.strip()
print(row_name, ":", row_value)

Wonderful! Let's write some code that can do this for all of the rows in the table we had above. 

In [ ]:
for row in rows:
    #Only process the rows with 4 cells to eliminate heading rows, etc.
    if len(row.find_all('td')) == 4:
        row_name = row.find_all('td')[0].text.strip()
        row_value = row.find_all('td')[1].text.strip() 
        print(row_name, ":", row_value)    

Whoa! We now have tangible data for January 1<sup>st</sup>. To make things simpler, let's use the 'Mean Temperature', 'Max Temperature', 'Min Temperature', 'Dew Point', 'Average Humidity', 'Maximum Humidity', 'Minimum Humidity', 'Precipitation', 'Wind Speed', 'Max Wind Speed', and 'Max Gust Speed' variables (also known as fields) from here out. Let's write a function that can scrape one HTML file given its name.

In [ ]:
fields = ['Mean Temperature', 'Max Temperature', 'Min Temperature',\
          'Dew Point', 'Average Humidity', 'Maximum Humidity',\
          'Minimum Humidity', 'Precipitation', 'Wind Speed',\
          'Max Wind Speed', 'Max Gust Speed']
def scrape_file(name):
    with open(name) as fin:
        soup = BeautifulSoup(fin.read(), "html.parser")
    data = {}
    for row in soup.find(id="historyTable").find_all("tr"):
        cells = row.find_all("td")
        if len(cells) == 4:
            name = cells[0].text.strip()
            if name in fields:
                data[name] = cells[1].text.split()[0].strip()   # Split to remove units
    return data
scrape_file("0.html")

Woo, we're making progress! Now that we can extract the data we want from any general HTML, it isn't too much more work to put together all of the data. We'll be storing all of this data in a special type of file called a *Comma Separated Values* (CSV) file; this just means a file that looks something like this:

```
A,B,C
1,2,3
5,10,15
```

It is essentially equivalent to a spreadsheet that is like this:

|  A  |  B  |  C  |
| :-: | :-: | :-: |
|  1  |  2  |  3  |
|  5  | 10  | 15  |

The useful thing about CSV files is that it is an extremely common data format that data science tools utilize. So common, in fact, that Python has a builtin tools to write them. Let's get started on writing out our CSV file!

In [ ]:
import csv

csv_fields = ["Month", "Day", "Year"] + fields

with open("weather_data.csv", "w") as fout:
    writer = csv.DictWriter(fout, csv_fields)
    writer.writeheader()
    
    for i, link in enumerate(links):
        if i % 50 == 0:
            print("Done with ", i)
        
        data = scrape_file("{}.html".format(i))
        url_parts = link.split("/")
        data["Month"] = int(url_parts[-3])
        data["Year"] = int(url_parts[-4])
        data["Day"] = int(url_parts[-2])
        
        writer.writerow(data)